In [ ]:
# pretrained classifier model
!pip install transformers tokenizers sentencepiece
!pip install mxnet
!pip install gluonnlp==0.8.0

# kobert
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

# NSMC
!git clone https://github.com/e9t/nsmc.git

In [1]:
# Data 불러오기
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/MyDrive/Colab Notebooks/OTT_team_project

/content/drive/MyDrive/Colab Notebooks/OTT_team_project


In [4]:
import utils.MODEL_utils_batch as model_utils

## 전처리 필요 라이브러리
import re
import os
import time
import datetime
import numpy as np
import pandas as pd

# 기타
from tqdm import tqdm, tqdm_notebook
import warnings # 경고 메시지 무시
import matplotlib.pyplot as plt
warnings.filterwarnings(action='ignore')

/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.
Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trai

## Main

keyword sentiment anlaysis

In [5]:
def predict_sentence(sentences):
  pred_kobert = []
  pred_koelectra = []
  # for s in tqdm_notebook(sentences):
  y_pred_kobert = model_utils.Custom_KoBERT_Predict(sentences)
  y_pred_koelectra = model_utils.Custom_KoELECTRA_Predict(sentences)

  pred_kobert.append(y_pred_kobert)
  pred_koelectra.append(y_pred_koelectra)
  # break

  return pred_kobert[0], pred_koelectra[0]

In [7]:
path = os.getcwd()+'/keyword_list'
sentiment_result = pd.DataFrame(columns = ['keyword', 'kobert_p', 'kobert_n', 'koelectra_p', 'koelectra_n', ])
for i, word in tqdm_notebook(enumerate(os.listdir(path))):
  sentences = pd.read_csv(path+'/'+word)
  sentences = list(sentences.loc[:, '0'])

  # 문장 예측
  pred_kobert, pred_koelectra = predict_sentence(sentences)

  # 비율 확인
  bert_p = pred_kobert.count(1) / len(pred_kobert)
  bert_n = pred_kobert.count(0) / len(pred_kobert)

  electra_p = pred_koelectra.count(1) / len(pred_koelectra)
  electra_n = pred_koelectra.count(0) / len(pred_koelectra)

  sentiment_result.loc[i] = [word[:-4], bert_p, bert_n, electra_p, electra_n]

  sentiment_result.to_csv('./data/SA_result.csv')

0it [00:00, ?it/s]

In [9]:
sentiment_result.to_csv('./data/SA_result.csv', encoding='utf8')

In [8]:
# 예시
sentiment_result

,keyword,kobert_p,kobert_n,koelectra_p,koelectra_n
0,돈,0.166031,0.833969,0.166667,0.833333
1,시리즈,0.664505,0.335495,0.444444,0.555556
2,자막,0.197519,0.802481,0.291667,0.708333
3,설치,0.160765,0.839235,0.000000,1.000000
4,인증,0.149378,0.850622,0.117647,0.882353
...,...,...,...,...,...
64,프로그램,0.626126,0.373874,0.666667,0.333333
65,화면,0.185419,0.814581,0.200000,0.800000
66,문제,0.194133,0.805867,0.076923,0.923077
67,라이브,0.647779,0.352221,0.500000,0.500000
